In [1]:
import os, sys, glob
import spacy
import pandas as pd

from spacy.pipeline import EntityRuler

In [2]:
sys.path.append('..')

In [3]:
from cycontext.item_data import ItemData
from cycontext.tag_object import TagObject
from cycontext.context_component import ConTextComponent

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
nlp = spacy.load("en_core_web_sm")
_ = nlp.remove_pipe("ner")

# Overview
- Read in data
- Read in modifiers

# I. Read in Knowledge Base

## I.I Targets

In [6]:
targets_filepath = os.path.join(".", "pneumonia", "kb", "pneumonia_targets.tsv")
targets_df = pd.read_csv(targets_filepath, sep="\t")

In [7]:
targets_df.head()

,Lex,Type,Regex,Direction,Codes
0,pneumonia,EVIDENCE_OF_PNEUMONIA,pneumonias?,NaN,NaN
1,consolidation,EVIDENCE_OF_PNEUMONIA,NaN,NaN,NaN
2,infiltrate,EVIDENCE_OF_PNEUMONIA,infiltrat(e|es|ion),NaN,NaN
3,effusion,EVIDENCE_OF_PNEUMONIA,effusion,NaN,NaN


In [8]:
target_patterns = []
for i, row in targets_df.fillna('').iterrows():
    literal = row["Lex"]
    category = row["Type"]
    regex = row["Regex"]
    
    if regex == '':
        pattern = [{"LOWER": literal}]
    else:
        pattern = [{"LOWER": {"REGEX": regex}}]
    target_patterns.append({"label": category, "pattern": pattern})
        

In [9]:
# Use the EntityRuler class for rule-based NER
ruler = EntityRuler(nlp, overwrite_ents=True)
ruler.add_patterns(target_patterns)

In [10]:
try:
    _ = nlp.remove_pipe("entity_ruler")
except:
    pass
nlp.add_pipe(ruler)

## I.II Modifiers

In [11]:
tsv_mods = pd.read_csv(os.path.join('.', "pneumonia", "kb", "pneumonia_modifiers.tsv"), sep="\t")

In [12]:
mod_patterns = []
for i, row in tsv_mods.fillna("").iterrows():
    d = {"literal": row["Lex"],
        "category": row["Type"],
        "pattern": row["Regex"],
        "rule": row["Direction"]}
    if d["pattern"] == "":
        d["pattern"] = None
    else:
        d["pattern"] = [d["pattern"]]
    mod_patterns.append(d)

In [13]:
import json

In [14]:
data = {"patterns": mod_patterns}

In [19]:
outpath = os.path.join(".", "pneumonia", "kb", "pneumonia_modifiers.json")
# with open(outpath, "w") as f:
#     json.dump(data, f, indent=4)

In [20]:
with open(outpath) as f:
    modifier_data = json.load(f)

In [21]:
modifier_data

{'patterns': [{'literal': 'are ruled out',
   'category': 'DEFINITE_NEGATED_EXISTENCE',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'be ruled out',
   'category': 'INDICATION',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'being ruled out',
   'category': 'INDICATION',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'can be ruled out',
   'category': 'DEFINITE_NEGATED_EXISTENCE',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'cannot be excluded',
   'category': 'AMBIVALENT_EXISTENCE',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'cannot totally be excluded',
   'category': 'PROBABLE_NEGATED_EXISTENCE',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'could be ruled out',
   'category': 'DEFINITE_NEGATED_EXISTENCE',
   'pattern': None,
   'rule': 'backward'},
  {'literal': 'examination',
   'category': 'INDICATION',
   'pattern': [{'LOWER': {'REGEX': '(examination|exam|study)'}}],
   'rule': 'backward'},
  {'literal

In [22]:
item_data = []
for data in modifier_data["patterns"]:
    item = ItemData(**data)
    item_data.append(item)

In [23]:
item_data

[ItemData: [are ruled out, definite_negated_existence, None, backward],
 ItemData: [be ruled out, indication, None, backward],
 ItemData: [being ruled out, indication, None, backward],
 ItemData: [can be ruled out, definite_negated_existence, None, backward],
 ItemData: [cannot be excluded, ambivalent_existence, None, backward],
 ItemData: [cannot totally be excluded, probable_negated_existence, None, backward],
 ItemData: [could be ruled out, definite_negated_existence, None, backward],
 ItemData: [examination, indication, [{'LOWER': {'REGEX': '(examination|exam|study)'}}], backward],
 ItemData: [free, definite_negated_existence, None, backward],
 ItemData: [has been ruled out, definite_negated_existence, None, backward],
 ItemData: [have been ruled out, definite_negated_existence, None, backward],
 ItemData: [is in the differential, ambivalent_existence, [{'LOWER': 'is'}, {'LOWER': 'in'}, {'LOWER': 'the'}, {'LOWER': 'differential'}], backward],
 ItemData: [is negative, definite_negat

In [36]:
context = ConTextComponent(item_data, nlp)

Pruning:  <TagObject> [can be ruled out, definite_negated_existence] <TagObject> [be ruled out, indication]
<TagObject> [can be ruled out, definite_negated_existence]

Pruning:  <TagObject> [cannot, ambivalent_existence] <TagObject> [cannot be excluded, ambivalent_existence]
<TagObject> [cannot be excluded, ambivalent_existence]

Pruning:  <TagObject> [cannot be excluded, ambivalent_existence] <TagObject> [not, definite_negated_existence]
<TagObject> [cannot be excluded, ambivalent_existence]

Pruning:  <TagObject> [cannot be excluded, ambivalent_existence] <TagObject> [cannot be excluded, ambivalent_existence]
<TagObject> [cannot be excluded, ambivalent_existence]

Pruning:  <TagObject> [cannot be excluded, ambivalent_existence] <TagObject> [not, definite_negated_existence]
<TagObject> [cannot be excluded, ambivalent_existence]

Pruning:  <TagObject> [cannot be excluded, ambivalent_existence] <TagObject> [cannot be excluded, ambivalent_existence]
<TagObject> [cannot be excluded, ambiv

Pruning:  <TagObject> [did rule her out against, definite_negated_existence] <TagObject> [did rule her out, definite_negated_existence]
<TagObject> [did rule her out against, definite_negated_existence]

Pruning:  <TagObject> [is to be ruled out, indication] <TagObject> [is to be ruled out for, indication]
<TagObject> [is to be ruled out for, indication]

Pruning:  <TagObject> [is to be ruled out for, indication] <TagObject> [be ruled out, indication]
<TagObject> [is to be ruled out for, indication]

Pruning:  <TagObject> [be ruled out, indication] <TagObject> [be ruled out for, indication]
<TagObject> [be ruled out for, indication]

Pruning:  <TagObject> [be ruled out for, indication] <TagObject> [ruled out, definite_negated_existence]
<TagObject> [be ruled out for, indication]

Pruning:  <TagObject> [is to be ruled out for, indication] <TagObject> [is to be ruled out for, indication]
<TagObject> [is to be ruled out for, indication]

Pruning:  <TagObject> [is to be ruled out for, indi

Pruning:  <TagObject> [not, definite_negated_existence] <TagObject> [not exhibit, definite_negated_existence]
<TagObject> [not exhibit, definite_negated_existence]

Pruning:  <TagObject> [not, definite_negated_existence] <TagObject> [not known to have, ambivalent_existence]
<TagObject> [not known to have, ambivalent_existence]

Pruning:  <TagObject> [ought to be ruled out, indication] <TagObject> [ought to be ruled out for, indication]
<TagObject> [ought to be ruled out for, indication]

Pruning:  <TagObject> [ought to be ruled out for, indication] <TagObject> [be ruled out, indication]
<TagObject> [ought to be ruled out for, indication]

Pruning:  <TagObject> [be ruled out, indication] <TagObject> [be ruled out for, indication]
<TagObject> [be ruled out for, indication]

Pruning:  <TagObject> [be ruled out for, indication] <TagObject> [ruled out, definite_negated_existence]
<TagObject> [be ruled out for, indication]

Pruning:  <TagObject> [ought to be ruled out for, indication] <TagOb

# II. Read Data

In [31]:
txt_files = glob.glob(os.path.join(".", "pneumonia", "training_v2", "*.txt"))

In [32]:
texts = []
for file in txt_files:
    texts.append(open(file).read())

# III. Process with NLP

In [37]:
try:
    nlp.add_pipe(context)
except:
    nlp.remove_pipe("context")
    nlp.add_pipe(context)

In [38]:
docs = []
for text in texts:
    docs.append(nlp(text))

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [increased, historical] <TagObject> [increased, historical]
<TagObject> [increased, historical]

Pruning:  <TagObject> [excluded, definite_negated_existence] <TagObject> [excluded, definite_negated_existence]
<TagObject> [excluded, definite_negated_existence]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR, conj] <TagObject> [REASON FOR THIS EXAMINATION, indica

Pruning:  <TagObject> [REASON FOR, conj] <TagObject> [REASON FOR THIS EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [No, definite_negated_existence] <TagObject> [No other, probable_negated_existence]
<TagObject> [No other, probable_negated_existence]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [prior study, historical] <TagObject> [stu

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR, conj] <TagObject> [REASON FOR THIS EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [No, definite_negated_existence] <TagObject> [No focal opacities are seen to suggest, definite_negated_existence]
<TagObject> [No focal opacities are seen to suggest, definite_negated_existence]

Pruning:  <TagObject> [No focal opacities are seen to suggest, definite_negated_existence] <TagObject> [suggest, probable_existence]
<TagObject> [No focal opacities are seen to suggest, definite_negated_existence]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR, conj] <TagObject> [REASON FOR THIS EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <TagObject> [EXAMINATION, indication]
<TagObject> [REASON FOR THIS EXAMINATION, indication]

Pruning:  <TagObject> [REASON FOR THIS EXAMINATION, indication] <

In [223]:
docs = list(nlp.pipe(texts))

In [39]:
for doc in docs:
    print(doc.ents)
    print(doc._.context_graph.modifiers)
    print(doc._.context_graph.edges)
    print()

(effusion, effusion)
[<TagObject> [old, historical], <TagObject> [REASON FOR, conj], <TagObject> [REASON FOR THIS EXAMINATION, indication], <TagObject> [EXAMINATION, indication], <TagObject> [INDICATION, indication], <TagObject> [examination, indication], <TagObject> [appear, probable_existence], <TagObject> [unchanged, historical], <TagObject> [unchanged, historical], <TagObject> [unchanged, historical], <TagObject> [again, historical], <TagObject> [No, definite_negated_existence], <TagObject> [Improving, historical], <TagObject> [stable, historical], <TagObject> [unchanged, historical]]
[(effusion, <TagObject> [unchanged, historical]), (effusion, <TagObject> [stable, historical])]

(consolidation, effusion, pneumonia)
[<TagObject> [old, historical], <TagObject> [REASON FOR, conj], <TagObject> [REASON FOR THIS EXAMINATION, indication], <TagObject> [EXAMINATION, indication], <TagObject> [INDICATION, indication], <TagObject> [consistent with, definite_existence], <TagObject> [unchanged,

In [40]:
pos_docs = [doc for doc in docs if len(doc.ents) > 0]

# Visualization

In [41]:
from cycontext import viz

In [42]:
from spacy import displacy

In [54]:
doc = docs[12]

In [55]:
viz.visualize_targets(doc
                     , colors ={"EVIDENCE_OF_PNEUMONIA": "orange",
                                          "DEFINITE_NEGATED_EXISTENCE": "blue"})

In [56]:
viz.visualize_modifiers(nlp("The patchy opacity in the left lower lobe could represent bronchopneumonia."))